In [1]:
import pandas as pd 
import numpy as np  

from sklearn.model_selection import train_test_split 
import tensorflow as tf 
from tensorflow import keras
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/Churn_Modelling_Cleaned.csv')
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,1,1,1,0.506735,1,False,False
1,0.516,0,0.310811,0.1,0.334031,1,0,1,0.562709,0,False,True
2,0.304,0,0.324324,0.8,0.636357,3,1,0,0.569654,1,False,False
3,0.698,0,0.283784,0.1,0.000000,2,0,0,0.469120,0,False,False
4,1.000,0,0.337838,0.2,0.500246,1,1,1,0.395400,0,False,True


In [3]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(20, input_shape=(11,), activation='relu'), # "more hidden layers can be added" 
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', 
              loss=loss, 
              metrics=['accuracy']) 
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)

    else: 
        model.fit(X_train, y_train, epochs=100, class_weights=weights)

    print("Model evaluation: \n", model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification report: \n", classification_report(y_test, y_preds))

    return y_preds

In [4]:
count_class_0, count_class_1 = df['Exited'].value_counts()

print('Count of class 0: ', count_class_0)
print('Count of class 1: ', count_class_1)

Count of class 0:  7963
Count of class 1:  2037


From above we can see that the customers who exited (class 0) has 7963 records whereas that of class 1 (didn't exit) is 2037. This is a significant difference creating biasness in the dataset.

### Applying "over sampling minority class using Synthetic Minority Over-sampling Technique (SMOTE)" to tackle imbalaced classes in data

SMOTE uses k-nearest neighbour algorithm to generate new samples out of current samples

In [5]:
X = df.drop('Exited', axis='columns')
y = df['Exited']

In [6]:
y.value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [7]:
from imblearn.over_sampling import SMOTE 

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X,y)

In [8]:
y_sm.value_counts()

Exited
1    7963
0    7963
Name: count, dtype: int64

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm) #"stratify=y" ensures that the class distribution in y_train and y_test is the same as in y

In [10]:
y_train.value_counts()

Exited
0    6370
1    6370
Name: count, dtype: int64

In [11]:
y_test.value_counts()

Exited
0    1593
1    1593
Name: count, dtype: int64

In [12]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5602 - loss: 0.6839
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6684 - loss: 0.6130
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7039 - loss: 0.5814
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7225 - loss: 0.5620
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7282 - loss: 0.5500
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7336 - loss: 0.5322
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7393 - loss: 0.5283
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7458 - loss: 0.5228
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7445 - loss: 0.5181
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7541 - loss: 0.5043
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7489 - loss: 0.5079
Epoch 12/100
399/399 ━━━━━━━━━━━━━━━━━━━━

Balanced performance – The model achieves similar performance for both classes, with precision and recall near identical for class 0 (78%, 77%) and class 1 (77%, 79%).

Class balance – The dataset is perfectly balanced, with 1593 instances of class 0 and 1593 instances of class 1, contributing to equal performance across both classes.

Moderate tradeoff – There is a slight preference for recall over precision for class 1, but the tradeoff is minimal, with both classes achieving an F1-score of 0.78.